# Transformer training (chantier en cours)

Imports

In [12]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from statapp.common.preprocessing import load_data, encode_data, split_into_X_y

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [14]:
text = load_data("data/fr.train.top1M.txt", sample=0.00001)

tokens = nltk.word_tokenize(text[:10])

In [15]:
len(tokens)

5

In [16]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [17]:
vocab_size

5

In [18]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

#Correspond à utiliser l'entropie croisée puisque les sorties sont des log_softmax
#et l'entropie croisée = nll_loss(log_softmax(.), target)
criterion = nn.NLLLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.01)

In [19]:
print(tokens_numbers)

[3 1 0 2 4]


In [20]:
print(tokens_numbers_sequences)

tensor([[3, 1, 0],
        [1, 0, 2],
        [0, 2, 4]])


In [21]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            print(output[:,:-1].reshape(-1, vocab_size))
            print(batch[:,1:].flatten())
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            #if i % 3 == 3:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1))
            running_loss = 0.

Test d'overfitting sur un cas ultrasimplifié (5 tokens, longueur de séquence 1) :
- En observant les sorties le modèle a bien appris et overfitte !

In [22]:
train_model(50,2)

C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.add(embedded, pos_encodings), dtype=torch.float32)


tensor([[-2.2521, -1.5649, -1.7163, -3.1983, -0.7654],
        [-2.8172, -2.4730, -2.4961, -3.8419, -0.2850],
        [-1.4964, -2.1916, -1.8331, -3.7179, -0.7337],
        [-2.5336, -3.6757, -1.7770, -3.6700, -0.3557]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 2, 4])
[1,     1] loss: 1.643
tensor([[-2.9091, -0.0676, -4.5918, -7.4537, -9.3510],
        [-0.5005, -1.1321, -2.6473, -7.7481, -8.8673],
        [-3.9742, -3.4602, -0.0518, -8.5958, -9.0302],
        [-2.2828, -2.3154, -0.5235, -4.7123, -1.6202]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 2, 4])
[2,     1] loss: 0.560
tensor([[-4.5767e+00, -5.0084e+00, -1.7404e-02, -8.7708e+00, -8.9631e+00],
        [-5.9339e+00, -6.1580e+00, -6.7752e+00, -5.3248e+00, -1.0834e-02],
        [-1.0329e-02, -5.3475e+00, -5.2241e+00, -8.9881e+00, -1.2010e+01],
        [-1.9755e+00, -3.9670e+00, -1.7290e-01, -6.9335e+00, -8.6358e+00]],
       grad_fn=<UnsafeViewBackward>)
tensor([2, 4, 0, 2])
[3,     1] loss: 0.053
tensor([[-8.02

       grad_fn=<UnsafeViewBackward>)
tensor([2, 4, 0, 2])
[22,     1] loss: 0.000
tensor([[-2.9070e+01, -2.9666e+01,  0.0000e+00, -2.1518e+01, -2.2145e+01],
        [-2.2136e+01, -2.6517e+01, -3.6294e+01, -1.9294e+01,  0.0000e+00],
        [-5.0996e+01,  0.0000e+00, -3.6123e+01, -3.4065e+01, -3.8437e+01],
        [-1.3351e-05, -1.4054e+01, -1.3637e+01, -1.1386e+01, -1.6795e+01]],
       grad_fn=<UnsafeViewBackward>)
tensor([2, 4, 1, 0])
[23,     1] loss: 0.000
tensor([[-5.2160e+01,  0.0000e+00, -3.6829e+01, -3.4653e+01, -3.9059e+01],
        [-1.1086e-05, -1.4429e+01, -1.4053e+01, -1.1547e+01, -1.6914e+01],
        [-6.1989e-06, -2.0798e+01, -1.2535e+01, -1.2855e+01, -1.9759e+01],
        [-2.8540e+01, -2.9423e+01,  0.0000e+00, -2.1343e+01, -2.3336e+01]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 0, 2])
[24,     1] loss: 0.000
tensor([[-3.0183e+01, -3.1065e+01,  0.0000e+00, -2.2627e+01, -2.3703e+01],
        [-2.2437e+01, -2.7081e+01, -3.7011e+01, -1.9758e+01,  0.0000e+00],
  

       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 0, 2])
[44,     1] loss: 0.000
tensor([[-6.3003e+01,  0.0000e+00, -4.3381e+01, -4.0010e+01, -4.4840e+01],
        [-1.4305e-06, -1.8334e+01, -1.7969e+01, -1.3460e+01, -1.8104e+01],
        [-3.5107e+01, -3.7298e+01,  0.0000e+00, -2.7681e+01, -3.0806e+01],
        [-2.3595e+01, -2.9459e+01, -3.9990e+01, -2.1748e+01,  0.0000e+00]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 2, 4])
[45,     1] loss: 0.000
tensor([[-3.5191e+01, -3.7405e+01,  0.0000e+00, -2.7770e+01, -3.0930e+01],
        [-2.3610e+01, -2.9498e+01, -4.0038e+01, -2.1782e+01,  0.0000e+00],
        [-6.3168e+01,  0.0000e+00, -4.3481e+01, -4.0089e+01, -4.4928e+01],
        [-1.3113e-06, -1.8402e+01, -1.8031e+01, -1.3500e+01, -1.8123e+01]],
       grad_fn=<UnsafeViewBackward>)
tensor([2, 4, 1, 0])
[46,     1] loss: 0.000
tensor([[-3.5267e+01, -3.7503e+01,  0.0000e+00, -2.7851e+01, -3.1043e+01],
        [-2.3622e+01, -2.9535e+01, -4.0082e+01, -2.1814e+01,  0.0000e+00],
  